<a href="https://colab.research.google.com/github/openjamoses/Model-conversion/blob/main/LeNet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch.nn import Module
from torch import nn

In [2]:
class Model(Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(256, 120)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(84, 10)
        self.relu5 = nn.ReLU()

    def forward(self, x):
        y = self.conv1(x)
        y = self.relu1(y)
        y = self.pool1(y)
        y = self.conv2(y)
        y = self.relu2(y)
        y = self.pool2(y)
        y = y.view(y.shape[0], -1)
        y = self.fc1(y)
        y = self.relu3(y)
        y = self.fc2(y)
        y = self.relu4(y)
        y = self.fc3(y)
        y = self.relu5(y)
        return y

In [3]:
import numpy as np
import torch
import csv
import time
import os
import copy
import csv
import pandas as pd
from datetime import datetime

from torchvision.datasets import mnist
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data_path = '/content/drive/MyDrive/Colab Notebooks/dataset/MNIST'
batch_size = 256
train_dataset = mnist.MNIST(root='/content/drive/MyDrive/Colab Notebooks/dataset/MNIST/train', train=True, transform=ToTensor())
test_dataset = mnist.MNIST(root=data_path+'/test', train=False, transform=ToTensor())
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
model = Model()
sgd = SGD(model.parameters(), lr=1e-1)
cost = CrossEntropyLoss()
epoch = 10  

model_name = 'lenet'

In [12]:
def train(model, num_epochs):
  since = time.time()
  date = datetime.today().strftime('%Y-%m-%d-%H:%M:%S')

  data_file = open('/content/exp_train_{}_{}.csv'.format(model_name, date), mode='w+', newline='', encoding='utf-8')
  data_writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  data_writer.writerow(['Model','Epoch','Train_loss', 'Train_acc','Val_loss', 'Val_acc', 'time','Elapse_time','date'])

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  #best_acc = 0
  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    since_1 = time.time()

    running_loss = 0.0
    running_corrects = 0
    _sum = 0
    since_1 = time.time()
    for idx, (train_x, train_label) in enumerate(train_loader):
        label_np = np.zeros((train_label.shape[0], 10))
        sgd.zero_grad()
        predict_y = model(train_x.float())
        loss = cost(predict_y, train_label.long())
        if idx % 10 == 0:
            print('idx: {}, loss: {}'.format(idx, loss.sum().item()))

        running_loss += loss.sum().item()
        predict_y = model(train_x.float()).detach()
        predict_ys = np.argmax(predict_y, axis=-1)
        label_np = train_label.numpy()
        _ = predict_ys == train_label
        running_corrects += np.sum(_.numpy(), axis=-1)
        _sum += _.shape[0]

        loss.backward()
        sgd.step()
    epoch_loss = running_loss / _sum
    epoch_acc = running_corrects / _sum
    
    
    #rows.append('{:.4f}'.format(epoch_loss))
    #rows.append('{:.4f}'.format(epoch_acc))


    correct = 0
    running_loss2 = 0.0
    _sum2 = 0

    for idx, (test_x, test_label) in enumerate(test_loader):
        
        predict_y = model(test_x.float())
        loss = cost(predict_y, test_label.long())
        predict_y = model(test_x.float()).detach()
        predict_ys = np.argmax(predict_y, axis=-1)
        label_np = test_label.numpy()
        _ = predict_ys == test_label
        correct += np.sum(_.numpy(), axis=-1)
        _sum2 += _.shape[0]
        running_loss2 += loss.sum().item()
        #loss.backward()

    epoch_loss2 = running_loss2 / _sum2
    epoch_acc2 = correct / _sum2
    print('accuracy: {:.2f}'.format(correct / _sum2))
    print('{} TrainLoss: {:.4f} TrainAcc: {:.4f}, ValLoss: {:.4f}, ValAcc: {:.4f}'.format(
                epoch, epoch_loss, epoch_acc, epoch_loss2, epoch_acc2))
    time_elapsed_1 = time.time() - since_1
    rows = [model_name, epoch, '{:.4f}'.format(epoch_loss), '{:.4f}'.format(epoch_acc),'{:.4f}'.format(epoch_loss2), '{:.4f}'.format(epoch_acc2), time.time(), 
            '{:.0f}m {:.0f}s'.format(time_elapsed_1 // 60, time_elapsed_1 % 60), date]
    data_writer.writerow(rows)
    val = correct / _sum
    if val > best_acc:
      model_best = model
      best_acc = val
      best_model_wts = copy.deepcopy(model.state_dict())

    #torch.save(model, '/content/mnist_{:.2f}.pkl'.format(correct / _sum))
  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))
  model.load_state_dict(best_model_wts)
  data_writer.writerow(['','', 'Best val Acc: {:4f}'.format(best_acc), time.time(),'Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60),''])

  data_file.close()
  return model 

In [14]:
model_ft = train(model,epoch)

Epoch 0/9
----------
idx: 0, loss: 1.0049935579299927
idx: 10, loss: 1.016793966293335
idx: 20, loss: 0.9878928065299988
idx: 30, loss: 0.9526361227035522
idx: 40, loss: 0.94521164894104
idx: 50, loss: 1.0548840761184692
idx: 60, loss: 0.9801884889602661
idx: 70, loss: 0.9798109531402588
idx: 80, loss: 1.0301663875579834
idx: 90, loss: 0.920755922794342
idx: 100, loss: 0.9329324960708618
idx: 110, loss: 0.9540637731552124
idx: 120, loss: 1.0282750129699707
idx: 130, loss: 0.9340881705284119
idx: 140, loss: 1.0027700662612915
idx: 150, loss: 0.8090428113937378
idx: 160, loss: 0.8769623041152954
idx: 170, loss: 0.8618240356445312
idx: 180, loss: 1.075975775718689
idx: 190, loss: 0.9355840086936951
idx: 200, loss: 0.9106934666633606
idx: 210, loss: 0.8904879689216614
idx: 220, loss: 0.9498329758644104
idx: 230, loss: 0.9810305237770081
accuracy: 0.58
0 TrainLoss: 0.0038 TrainAcc: 0.5902, ValLoss: 0.0039, ValAcc: 0.5850
Epoch 1/9
----------
idx: 0, loss: 1.001354694366455
idx: 10, loss: 1.

In [15]:
model_ft

Model(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (relu4): ReLU()
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu5): ReLU()
)

In [ ]:
!pip install tensorflow
!pip install onnx==1.8.1
!pip install onnx_tf
!pip install onnx_pytorch
!pip install pytorch2keras
#%tensorflow_version 1.x
from tensorflow import keras
import tensorflow as tf
print(tf.__version__)

In [17]:
#Import needed packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from onnx_tf.backend import prepare
from __future__ import print_function, division
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
from pytorch2keras.converter import pytorch_to_keras
import matplotlib.pyplot as plt
import time
import os
import copy
import csv
import pandas as pd
from datetime import datetime

plt.ion()   # interactive mode

In [19]:
#model_ft = torch.load('/content/mnist_0.69.pkl')

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
for idx, (train_x, train_label) in enumerate(train_loader):
  print(train_x.size(),train_label.size())

In [19]:
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
feature_extract = True
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 conv1.weight
	 conv1.bias
	 conv2.weight
	 conv2.bias
	 fc1.weight
	 fc1.bias
	 fc2.weight
	 fc2.bias
	 fc3.weight
	 fc3.bias


In [22]:
input_np = np.random.uniform(0, 1, (1, 1, 28, 28))
input_var = Variable(torch.FloatTensor(input_np))
k_model_ft = pytorch_to_keras(model_ft, input_var, [(1, 28, 28,)], verbose=True, change_ordering=True)

INFO:pytorch2keras:Converter is called.
DEBUG:pytorch2keras:Input_names:
DEBUG:pytorch2keras:['input_0']
DEBUG:pytorch2keras:Output_names:
DEBUG:pytorch2keras:['output_0']
INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:[(1, 28, 28)]
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output_0.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight conv1.weight with shape (6, 1, 5, 5).
DEBUG:onnx2keras:Found weight conv1.bias with shape (6,).
DEBUG:onnx2keras:Found weight conv2.weight with shape (16, 6, 5, 5).
DEBUG:onnx2keras:Found weight conv2.bias with shape (16,).
DEBUG:onnx2keras:Found weight fc1.weight with shape (120, 256).
DEBUG:onnx2keras:Found weight fc1.bias with shape (120,).
DEBUG:onnx2keras:Found weight fc2.weight with shape (84, 120).
DEBUG:onnx2keras:Found weight fc2.bias with shape (84,).
DEBUG:onnx2keras:Found weight fc3.weight 

graph(%input_0 : Float(1, 1, 28, 28, strides=[784, 784, 28, 1], requires_grad=0, device=cpu),
      %conv1.weight : Float(6, 1, 5, 5, strides=[25, 25, 5, 1], requires_grad=1, device=cpu),
      %conv1.bias : Float(6, strides=[1], requires_grad=1, device=cpu),
      %conv2.weight : Float(16, 6, 5, 5, strides=[150, 25, 5, 1], requires_grad=1, device=cpu),
      %conv2.bias : Float(16, strides=[1], requires_grad=1, device=cpu),
      %fc1.weight : Float(120, 256, strides=[256, 1], requires_grad=1, device=cpu),
      %fc1.bias : Float(120, strides=[1], requires_grad=1, device=cpu),
      %fc2.weight : Float(84, 120, strides=[120, 1], requires_grad=1, device=cpu),
      %fc2.bias : Float(84, strides=[1], requires_grad=1, device=cpu),
      %fc3.weight : Float(10, 84, strides=[84, 1], requires_grad=1, device=cpu),
      %fc3.bias : Float(10, strides=[1], requires_grad=1, device=cpu)):
  %11 : Float(1, 6, 24, 24, strides=[3456, 576, 24, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=

DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 6, 24, 24), dtype=tf.float32, name=None), name='11/BiasAdd:0', description="created by layer '11'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Relu
DEBUG:onnx2keras:node_name: 12
DEBUG:onnx2keras:node_params: {'change_ordering': True, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 11).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 6, 24, 24), dtype=tf.float32, name=None), name='12/Relu:0', description="created by layer '12'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: MaxPool
DEBUG:onnx2keras:node_name: 13
DEBUG:onnx2keras:node_params: {'kernel_shape': [2, 2], 'pads': [0, 0, 0, 0], 'strides': [2, 2], 'change_order

In [23]:
k_model_ft.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_0 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
11 (Conv2D)                  (None, 24, 24, 6)         156       
_________________________________________________________________
12 (Activation)              (None, 24, 24, 6)         0         
_________________________________________________________________
13 (MaxPooling2D)            (None, 12, 12, 6)         0         
_________________________________________________________________
14 (Conv2D)                  (None, 8, 8, 16)          2416      
_________________________________________________________________
15 (Activation)              (None, 8, 8, 16)          0         
_________________________________________________________________
16 (MaxPooling2D)            (None, 4, 4, 16)          0     

In [24]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# Rescale the images from [0,255] to the [0.0,1.0] range.
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_test))

11493376/11490434 [==============================] - 0s 0us/step
Number of original training examples: 60000
Number of original test examples: 10000


In [25]:
print(tf.shape(x_train))

tf.Tensor([60000    28    28     1], shape=(4,), dtype=int32)


In [26]:
k_model_ft.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [27]:
k_model_ft_history = k_model_ft.fit(x_train,y_train, epochs=epoch)

Epoch 1/10
1875/1875 [==============================] - 36s 3ms/step - loss: 0.9791 - accuracy: 0.5864
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.9683 - accuracy: 0.5886
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.9604 - accuracy: 0.5899
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.9572 - accuracy: 0.5902
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.9528 - accuracy: 0.5912
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.9500 - accuracy: 0.5917
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.9482 - accuracy: 0.5915
Epoch 8/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.9468 - accuracy: 0.5922
Epoch 9/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.9453 - accuracy: 0.5926
Epoch 10/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.94

In [28]:
def export_history_csv2(history_):
  since = time.time()
  date = datetime.today().strftime('%Y-%m-%d-%H:%M:%S')
  data_file = open('/content/tensorflow_exp_train_{}.csv'.format(date), mode='w+', newline='', encoding='utf-8')
  data_writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  data_writer.writerow(['Model','type', 'Dataset', 'Epoch', 'criterion', 'optimizer', 'scheduler','Train_loss', 'Train_acc', 'time','Elapse_time','date'])
  for epoch_ in history_.epoch:
    data_writer.writerow([history_.model,'tensorflow', 'hymenoptera', epoch_, '', 
                          history_.model.optimizer, '',history_.history['loss'][epoch_], history_.history['accuracy'][epoch_], 
                           '','',date])
  data_file.close()

In [29]:
export_history_csv2(k_model_ft_history)

In [30]:
# Input to the model
model_name = 'lenet5'
torch.random.manual_seed(42)
x = torch.randn(batch_size, 1, 28, 28, requires_grad=True)
torch_out = model_ft(x)
# Export the model
torch.onnx.export(model_ft,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "model_ft-{}.onnx".format(model_name),   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [31]:
import onnx
onnx_model = onnx.load("model_ft-{}.onnx".format(model_name))
onnx.checker.check_model(onnx_model)

In [32]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("model_ft-{}.onnx".format(model_name))

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


In [33]:
np.testing.assert_array_equal(to_numpy(torch_out), ort_outs[0])

AssertionError: ignored

In [34]:
from onnx2keras import onnx_to_keras
k_model_keras = onnx_to_keras(onnx_model, ['input'])

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight conv1.weight with shape (6, 1, 5, 5).
DEBUG:onnx2keras:Found weight conv1.bias with shape (6,).
DEBUG:onnx2keras:Found weight conv2.weight with shape (16, 6, 5, 5).
DEBUG:onnx2keras:Found weight conv2.bias with shape (16,).
DEBUG:onnx2keras:Found weight fc1.weight with shape (120, 256).
DEBUG:onnx2keras:Found weight fc1.bias with shape (120,).
DEBUG:onnx2keras:Found weight fc2.weight with shape (84, 120).
DEBUG:onnx2keras:Found weight fc2.bias with shape (84,).
DEBUG:onnx2keras:Found weight fc3.weight with shape (10, 84).
DEBUG:onnx2keras:Found weight fc3.bias with shape (10,).
DEBUG:onnx2keras:Found weight 31 with shape (1,).
DEBUG:onnx2keras:Found input input with shape [1, 28, 28]

In [35]:
x_train2 = tf.reshape(x_train, [-1,1, 28,28])
x_test2 = tf.reshape(x_test, [-1,1, 28,28])

In [36]:
pred_k_keras = k_model_keras.predict(x_test2)

In [37]:
# serialize model to JSON
def save_keras(model, model_type='direct'):
  model_json = model.to_json()
  with open("k_model_{}_{}.json".format(model_name, model_type), "w") as json_file:
    json_file.write(model_json)
  # serialize weights to HDF5
  model.save_weights("k_model_{}_{}.h5".format(model_name, model_type))
  print("Saved model to disk")

In [38]:
save_keras(k_model_ft, 'direct')
save_keras(k_model_keras, 'through_onnx')

DEBUG:h5py._conv:Creating converter from 5 to 3


Saved model to disk
Saved model to disk


In [ ]:
!pip install -U tf2onnx

In [40]:
import tf2onnx

model_proto, external_tensor_storage = tf2onnx.convert.from_keras(k_model_keras,
                input_signature=None, opset=None, custom_ops=None,
                custom_op_handlers=None, custom_rewriter=None,
                inputs_as_nchw=None, extra_opset=None, shape_override=None,
                 target=None, large_model=False, output_path='model_ft_keras-{}2.onnx'.format(model_name))

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tf2onnx.tfonnx:Using tensorflow=2.5.0, onnx=1.8.1, tf2onnx=1.9.1/8e8c23
INFO:tf2onnx.tfonnx:Using opset <onnx, 9>
INFO:tf2onnx.tf_utils:Computed 0 values for constant folding
DEBUG:tf2onnx.graph:Making node: Name=Identity, OP=Identity
DEBUG:tf2onnx.graph:Made node: Identity
OP=Identity
Name=Identity
Inputs:
	model_1/output/Relu:0=Relu, [-1, 10], 1
Outpus:
	Identity_raw_output___4:0=[-1, 10], 1
DEBUG:tf2onnx.graph:Making node: Name=Identity_graph_outputs_Identity__5, OP=Identity
DEBUG:tf2onnx.graph:Infer shape and dtype for [Identity_graph_outputs_Identity__5]
DEBUG:tf2onnx.graph:Set dtype of [output] to 1
DEBUG:tf2onnx.graph:Set shape of [output] to [-1, 10]
DEBUG:tf2onnx.graph:Made node: Identity_graph_outputs_Identity__5
OP=Identity
Name=Identity_graph_outputs_Identity__5
Inputs:
	Identity_raw_output___4:0=Identity, [-1, 10], 1
Outpus:
	output=[-1, 10], 1
DEBUG:tf2onnx.rewriter.lstm_rewriter:enter lstm re

In [41]:
onnx_model_keras = onnx.load('model_ft_keras-{}2.onnx'.format(model_name))
onnx.checker.check_model(onnx_model_keras)

In [42]:
import onnxruntime

ort_session2 = onnxruntime.InferenceSession('model_ft_keras-{}2.onnx'.format(model_name))
# compute ONNX Runtime output prediction
ort_inputs2 = {ort_session2.get_inputs()[0].name: to_numpy(x)}
ort_outs2 = ort_session2.run(None, ort_inputs)

In [43]:
np.testing.assert_array_equal(ort_outs2[0], ort_outs[0])

AssertionError: ignored

In [44]:
model_proto, external_tensor_storage = tf2onnx.convert.from_keras(k_model_ft,
                input_signature=None, opset=None, custom_ops=None,
                custom_op_handlers=None, custom_rewriter=None,
                inputs_as_nchw=None, extra_opset=None, shape_override=None,
                 target=None, large_model=False, output_path='model_ft-{}2.onnx'.format(model_name))

INFO:tf2onnx.tfonnx:Using tensorflow=2.5.0, onnx=1.8.1, tf2onnx=1.9.1/8e8c23
INFO:tf2onnx.tfonnx:Using opset <onnx, 9>
INFO:tf2onnx.tf_utils:Computed 0 values for constant folding
DEBUG:tf2onnx.graph:Making node: Name=Identity, OP=Identity
DEBUG:tf2onnx.graph:Made node: Identity
OP=Identity
Name=Identity
Inputs:
	model/output_0/Relu:0=Relu, [-1, 10], 1
Outpus:
	Identity_raw_output___12:0=[-1, 10], 1
DEBUG:tf2onnx.graph:Making node: Name=Identity_graph_outputs_Identity__13, OP=Identity
DEBUG:tf2onnx.graph:Infer shape and dtype for [Identity_graph_outputs_Identity__13]
DEBUG:tf2onnx.graph:Set dtype of [output_0] to 1
DEBUG:tf2onnx.graph:Set shape of [output_0] to [-1, 10]
DEBUG:tf2onnx.graph:Made node: Identity_graph_outputs_Identity__13
OP=Identity
Name=Identity_graph_outputs_Identity__13
Inputs:
	Identity_raw_output___12:0=Identity, [-1, 10], 1
Outpus:
	output_0=[-1, 10], 1
DEBUG:tf2onnx.tfonnx:op name model/24/Reshape/shape, 2, 0
DEBUG:tf2onnx.graph:Making node: Name=Flatten__15, OP=F

In [45]:
onnx_model_22 = onnx.load('model_ft-{}2.onnx'.format(model_name))
onnx.checker.check_model(onnx_model_22)
ort_session22 = onnxruntime.InferenceSession('model_ft-{}2.onnx'.format(model_name))
# compute ONNX Runtime output prediction
ort_inputs22 = {ort_session22.get_inputs()[0].name: to_numpy(x).reshape(-1,28,28,1)}
ort_outs22 = ort_session22.run(None, ort_inputs22)

In [46]:
!pip install onnx-pytorch

In [47]:
from onnx_pytorch import code_gen
code_gen.gen("/content/model_ft-{}2.onnx".format(model_name), "/content")

# Autogenerated by onnx-pytorch.

import glob
import os

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.__vars = nn.ParameterDict()
    for b in glob.glob(
        os.path.join(os.path.dirname(__file__), "variables", "*.npy")):
      v = torch.from_numpy(np.load(b))
      requires_grad = v.dtype.is_floating_point or v.dtype.is_complex
      self.__vars[os.path.basename(b)[:-4]] = nn.Parameter(
          torch.from_numpy(np.load(b)), requires_grad=requires_grad)
    self.n_n_model_11_BiasAdd = nn.Conv2d(**{'groups': 1, 'dilation': [1, 1], 'out_channels': 6, 'padding': 0, 'kernel_size': (5, 5), 'stride': [1, 1], 'in_channels': 1, 'bias': True})
    self.n_n_model_11_BiasAdd.weight.data = self.__vars["t_model_11_Conv2D_ReadVariableOp_0"]
    self.n_n_model_11_BiasAdd.bias.data = self.__vars["t_model_11_BiasAdd_ReadVariableOp_0"]
    self.n_n_model_13_MaxPool = 

In [48]:
onnx_model_tf = onnx.load('/content/model_ft-{}2.onnx'.format(model_name))
tf_rep = prepare(onnx_model_tf)  # prepare tf representation
tf_rep.inputs

['input_0']

In [49]:
import numpy as np
import onnx
import onnxruntime
import torch

torch.set_printoptions(8)

from model import Model

In [56]:
np.random.seed(42)
model = Model()
model.eval()
#inp = np.random.randn(batch_size, 1, 28, 28).astype(np.float32)
torch.random.manual_seed(42)
x = torch.randn(batch_size, 1, 28, 28, requires_grad=True)
#torch_out = model_ft(x)

with torch.no_grad():
  #torch_outputs = model(torch.from_numpy(inp))
  torch_outputs = model(x)
  

onnx_model = onnx.load("/content/model_ft-{}2.onnx".format(model_name))
sess_options = onnxruntime.SessionOptions()
session = onnxruntime.InferenceSession(onnx_model.SerializeToString(),
                                       sess_options)
#print(session.get_inputs)

ort_session2 = onnxruntime.InferenceSession('model_ft_keras-{}2.onnx'.format(model_name))
# compute ONNX Runtime output prediction
ort_inputs = {ort_session2.get_inputs()[0].name: to_numpy(x)}
ort_outputs = ort_session2.run(None, ort_inputs)

print(
    "Comparison result:",
    np.allclose(torch_outputs.detach().numpy(),
                ort_outputs[0],
                atol=1e-5,
                rtol=1e-5))

Comparison result: False


In [51]:
np.testing.assert_array_equal(torch_outputs.detach().numpy(), ort_outputs[0])

AssertionError: ignored

In [53]:
!pip install mxnet

     |████████████████████████████████| 46.9 MB 39 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [54]:
from PIL import Image
import numpy as np
import mxnet as mx
import mxnet.contrib.onnx as onnx_mxnet
from mxnet.test_utils import download
from matplotlib.pyplot import imshow

In [58]:
#onnx_model_file = mxnet(model_url, '/content/model_ft-{}2.onnx')
sym, arg_params, aux_params = onnx_mxnet.import_model("/content/model_ft-{}2.onnx".format(model_name))

KeyError: ignored

In [ ]:
mx.viz.plot_network(sym, node_attrs={"shape":"oval","fixedsize":"false"})